In [ ]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

In [ ]:
import sys

sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
#offload_per_layer = 4
offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
text="""Samyak Jain
PhNo9557619919 jainsamyak9615@gmail.com LinkedIn Github
EDUCATION
 Teerthanker Mahaveer University, Moradabad Bachelor in Computer Application, Jul 2021 – May 2024
 M.D Jain Inter College, 12th — UP, Mar 2020 – Apr 2021
 M.D Jain Inter College, 10th — UP, Mar 2018 – Apr 2019
Online Courses: Predictive Analytics for Business – Data Analytics (Coursera Google Data Analytics Certificate), Data Science Masters
(Pwskills), and Machine Learning (iNeuron.ai)
WORK EXPERIENCE
Oasis Infobyte – Email Spam Detection Data Science Intern April 2023 – May 2023
● Implemented a Machine Learning – (Machine Learning) model using a Naïve Bayes algorithm to predict email spam. By
analyzing the contents and the features of emails as a spam or non-spam, achieving high accuracy and assisting in filtering
unwanted email.
NoQs Digital – Diversity and Inclusion Analysis Dashboard May 2023 –July 2023
Data Analyst Intern
 “Analyzed (Power BI) employee performance by gender, percentage of employees in different time types, FY20 leaver statistics,
country-wise employee distribution, and gender ratio in the workforce.
PROJECTS
Quora Question Pair (GitHub Link)
● Utilizing Natural Language Processing techniques (Tokenization, Stopwords, Lemmatization, and Word-embedding) to identify
duplicate questions on Quora by analyzing semantic similarity.
● Implementing machine learning algorithm Random Forest and Xgboost and leveraged Streamlit to craft an intuitive user
interface for seamless interaction with the NLP model.
Algerian Forest Fire (GitHub Link)
● Developed a that accurately predict the potential damage caused by forest fire using Logistic Regression, identify the key variable
that influenced the severity of forest fires.
● Utilized Flask to develop a streamlined user interface for the model, ensuring a responsive and seamless experience.
Implemented features for effective interaction and user engagement.
Car Price Prediction (GitHub Link)
● Applied Linear Regression to model the relationship between car features and pricing. Conducted through data analysis, and
feature engineering, and implemented a predictive model to accurately estimate car prices, enhancing decision-making for buyers
and sellers in the automotive market.
● Utilized python and popular libraries like Pandas, Scikit-Learn, and Matplotlib for implementation.
SKILLS
● Programming Languages: Python(Pandas, Numpy, Matplotlib, and Sklearn) and SQL.
● Data Analysis and Visualization: Power BI.
● Data Analysis and Visualization: Supervised, Unsupervised, and Clustering.
● Natural Language Processing: Tokenization, Lemmatization, and Word-Embedding e.t.c
● Data Wrangling: Exploratory Data Analysis, Feature Engineering.
● Mathematics: Statistics and Probability.
● Tools and Platforms: Google Collab, Jupyter Notebook, and Github.
ACHIEVEMENTS
● 3rd Runner-up in KIMO’s –Edge 2023.
● Attended the International Conference on AWS which was held by Teerthanker Mahaveer University, Moradabad.
● As a Campus Ambassador, I successfully implemented effective promotional strategies, coordinated events, and enhanced
brand visibility. Showcased strong communication and organizational skills to drive increased campus engagement"""

In [ ]:
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0

# User input
user_input = text + ("  what are the projects mentioned in the above text? Only give the name of the projects don't add any other text")

# Tokenize user input
user_entry = dict(role="user", content=user_input)
input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

if past_key_values is None:
    attention_mask = torch.ones_like(input_ids)
else:
    seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
    attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

# Generate response
result = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    past_key_values=past_key_values,
    streamer=streamer,
    do_sample=True,
    temperature=0.9,
    top_p=0.9,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
    output_hidden_states=True,
)

# Decode generated text
generated_text = tokenizer.decode(result["sequences"][0], skip_special_tokens=True)
print(generated_text)


The projects mentioned in the text are:

* Quora Question Pair
* Algerian Forest Fire
* Car Price Prediction
[INST] Samyak Jain
PhNo9557619919 jainsamyak9615@gmail.com LinkedIn Github
EDUCATION
 Teerthanker Mahaveer University, Moradabad Bachelor in Computer Application, Jul 2021 – May 2024
 M.D Jain Inter College, 12th — UP, Mar 2020 – Apr 2021
 M.D Jain Inter College, 10th — UP, Mar 2018 – Apr 2019
Online Courses: Predictive Analytics for Business – Data Analytics (Coursera Google Data Analytics Certificate), Data Science Masters
(Pwskills), and Machine Learning (iNeuron.ai)
WORK EXPERIENCE
Oasis Infobyte – Email Spam Detection Data Science Intern April 2023 – May 2023
● Implemented a Machine Learning – (Machine Learning) model using a Naïve Bayes algorithm to predict email spam. By
analyzing the contents and the features of emails as a spam or non-spam, achieving high accuracy and assisting in filtering
unwanted email.
NoQs Digital – Diversity and Inclusion Analysis Dashboard May

In [ ]:
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0

# User input
user_input = text + ("  what are the projects mentioned in the above text? Only give the name of the projects don't add any other text")

# Tokenize user input
user_entry = dict(role="user", content=user_input)
input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

if past_key_values is None:
    attention_mask = torch.ones_like(input_ids)
else:
    seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
    attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

# Generate response
result = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    past_key_values=past_key_values,
    streamer=streamer,
    do_sample=True,
    temperature=0.9,
    top_p=0.9,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
    output_hidden_states=True,
)

# Get generated text without user input
generated_text = tokenizer.decode(result["sequences"][0], skip_special_tokens=True)
# Remove user input from the generated text
generated_text = generated_text[len(user_input):].strip()
print(generated_text)


The projects mentioned in the text are:

* Quora Question Pair
* Algerian Forest Fire
* Car Price Prediction
er text [/INST] The projects mentioned in the text are:

* Quora Question Pair
* Algerian Forest Fire
* Car Price Prediction


In [ ]:
from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0
while True:
  #print("User: ", end="")
  user_input =text+("  what is the phone number mentioned in the above text ?Only give the phone number don't add any other text")
  #print("\n")

  user_entry = dict(role="user", content=user_input)
  input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

  if past_key_values is None:
    attention_mask = torch.ones_like(input_ids)
  else:
    seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
    attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

  print("Mixtral: ", end="")
  result = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    past_key_values=past_key_values,
    streamer=streamer,
    do_sample=True,
    temperature=0.9,
    top_p=0.9,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
    output_hidden_states=True,
  )
  print("\n")

  phone_number = result["sequences"]
  past_key_values = result["past_key_values"]
  print(phone_number)

In [ ]:
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0

# User input
user_input = text + ("  what is the phone number mentioned in the above text ?Only give the phone number don't add any other text. Do not write any sentence")

# Tokenize user input
user_entry = dict(role="user", content=user_input)
input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

if past_key_values is None:
    attention_mask = torch.ones_like(input_ids)
else:
    seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
    attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

# Generate response
result = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    past_key_values=past_key_values,
    streamer=streamer,
    do_sample=True,
    temperature=0.9,
    top_p=0.9,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
    output_hidden_states=True,
)

# Get generated text without user input
generated_text = tokenizer.decode(result["sequences"][0], skip_special_tokens=True)

# Find index of "[/INST]"
inst_index = generated_text.rfind("[/INST]")

# Remove everything from "[/INST]" onwards
if inst_index != -1:  # Check if "[/INST]" is found
    generated_text = generated_text[:inst_index]

# Remove user input from the generated text
generated_text = generated_text[len(user_input):].strip()
print(generated_text)



The phone number mentioned in the above text is 